In [ ]:
import os
import pandas as pd
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.factorplots import interaction_plot
import numpy as np
from scipy import stats

In [ ]:
# load the data
search_path = os.getcwd() + '\\data\\' # location of the data files
df = pd.read_csv(search_path + "data_questionnaires_preprocessed.csv", converters={'BE04_01':str, 'BE04_02':str, 'BE04_03':str})
df = df.rename(columns={"blob" : 'condition'})

In [ ]:
sig_level = 0.05 # p value to be used as threshold

## Notes on Mixed ANOVA

**Independent variables:** two categorical independent variables (factors)
- one of which is between-subject (each subjects assigned only once to treatment): blob vs. avatar
- the other is within-subject (each subject assigned multiple treatments): hybrid vs. first-person

**Assumptions:**
- The responses from subjects (dependent variable) should be continuous
- Residuals (experimental error) are approximately normally distributed for each combination of between-subject and within-subject variable (Shapiro-Wilks Test or histogram)
- Homogeneity of variances or homoscedasticity: There should be equal variance for every level of within-subject factor (Levene’s test)
- Assumption of sphericity: the variances of differences in responses between any two levels of the independent variable (within-subjects factor) should be equal (Mauchly’s test of sphericity). This assumptionn is also known as homogeneity-of-variance-of-differences assumption.
- Homogeneity of the variance-covariance matrices: the pattern of intercorrelation for each level of within-subject variable across between-subject variable should be equal. This is a multivariate version of the Homogeneity of variances. It can be tested using Box’s M test. Box’s M-test has little power and uses a lower alpha level such as 0.001 to assess the p value for significance.
- There should be no significant outlier (this can be checked by boxplot)
 
**Results:**
- Two-way mixed ANOVA estimates the three effects - two main effects and one interaction effect - for statistical significance
- Generally, it is not appropriate to interpret main effects when interaction is significant.

(Source: https://www.reneshbedre.com/blog/mixed-anova.html)


## Naming conventions

- condition: blob vs. avatar (1: blob, 0: avatar)
- mode: first-person vs. hybrid

## Functions

In [ ]:
def summarize(df, var):
    '''
    Groups the df by the independent factors, and prints the variable's count, mean, and std.
    
    df (pd.DataFrame)
    var (str): the column to summarize
    '''
    return df.groupby(["condition", "mode"]).describe()[var][["count", "mean", "std"]]

In [ ]:
def boxplot(df, var, title, ax=None):
    '''
    Creates a boxplot of the variable, separated by condition and mode.
    
    df (pd.DataFrame)
    var (str): the column to plot
    title (str): title of the boxplot
    '''
    if not ax:
        sns.boxplot(x='condition', y=var, hue='mode', data=df)
        plt.suptitle(title)
    else:
        sns.boxplot(x='condition', y=var, hue='mode', data=df, ax=ax)
        ax.set_title(title, fontsize=14)
    sns.despine()

In [ ]:
def interact_plot(df, var, title, ax=None):
    '''
    Plots the interaction of condition and mode for the variable.
    
    df (pd.DataFrame)
    var (str):  dependent variable column name
    title (str): title of the plot
    '''
    if not ax: 
        fig = interaction_plot(x=df['condition'], trace=df['mode'], response=df[var], colors=['#4c061d','#d17a22'])
        fig.suptitle(title)
    else:
        interaction_plot(x=df['condition'], trace=df['mode'], response=df[var], colors=['#4c061d','#d17a22'], ax=ax)
        ax.set_title(title, fontsize=14)
    sns.despine()
    plt.show()

In [ ]:
def check_assumptions(df, dv, dv_name):
    '''
    Checks the assumptions of a mixed ANOVA.
    
    df (pd.DataFrame)
    dv (str): dependent variable column name
    dv_name (str): name of the dependent variable (to be printed)
    '''
    
    print("Checking the assumptions for the dependent variable {}...\n".format(dv_name))
    all_assumptions = []
    
    # Assumption of the residuals being normally distributed
    df['factor_comb'] = df["condition"] + '-'+ df["mode"]
    normal = pg.normality(data=df, dv=dv, group='factor_comb')
    if normal["normal"].all():
        print("The residuals are approximately normally distributed for each level of the within-subjects factor (tested using the Shapiro-Wilks Test)")
        all_assumptions.append(True)
    else:
        print("The residuals are NOT approximately normally distributed for each level of the within-subjects factor! (tested using the Shapiro-Wilks Test)")
        all_assumptions.append(False)
    
    print()
    # Assumption of homoscedasticity
    print("There should be equal variance for every level of within-subject factor:")
    hybrid = df[df["mode"]=="Hybrid"].reset_index(drop=True)
    homo_hybrid = pg.homoscedasticity(data=hybrid, dv=dv, group='condition')
    all_assumptions.append(homo_hybrid["equal_var"][0])
    print("For the level Hybrid, Levene's test resulted in a p-value of {}, indicating that the assumption of equal variances is {}.".format(homo_hybrid["pval"][0].round(3), homo_hybrid["equal_var"][0]))
    
    fp = df[df["mode"]=="First Person"].reset_index(drop=True)
    homo_fp = pg.homoscedasticity(data=fp, dv=dv, group='condition')
    all_assumptions.append(homo_fp["equal_var"][0])
    print("For the level First Person, Levene's test resulted in a p-value of {}, indicating that the assumption of equal variances is {}.".format(homo_fp["pval"][0].round(3), homo_fp["equal_var"][0]))
    
    print()    
    # Assumption of sphericity
    sphericity = pg.sphericity(data=df, dv=dv, subject='ID', within='condition')
    all_assumptions.append(sphericity[0])
    print("Mauchly’s test of sphericity resulted in a p-value of {}, indicating that the assumption is {}.".format(sphericity[-1], sphericity[0]))
    
    print()
    # Assumption of homogeneity of the variance-covariance matrices:
    print("The homogeneity of variance-covariance matrices formed by the between-subject factor for each level of the within-subject factor should be equal:")
    box_fp = pg.box_m(data=fp, dvs=[dv], group='condition', alpha=0.001)
    all_assumptions.append(box_fp["equal_cov"][0])    
    print("For the level First Person, Box’s M test resulted in a p-value of {}, indicating that the assumption is {}.".format(box_fp["pval"][0].round(3), box_fp["equal_cov"][0]))
    
    box_hybrid = pg.box_m(data=hybrid, dvs=[dv], group='condition', alpha=0.001)
    all_assumptions.append(box_hybrid["equal_cov"][0])    
    print("For the level Hybrid, Box’s M test resulted in a p-value of {}, indicating that the assumption is {}.".format(box_hybrid["pval"][0].round(3), box_hybrid["equal_cov"][0]))

    print()
    # Assumption of no outliers
    outliers = df[df[dv] > df[dv].mean() + 3 * df[dv].std()]
    if outliers.empty:
        print("There are no outliers!")
        all_assumptions.append(True)
    else: 
        print("There are outliers! The outliers are the participants {}".format(outliers["ID"].values))
        all_assumptions.append(False)
    
    
    if all(x == True for x in all_assumptions):
        print()
        print("ALL ASSUMPTIONS ARE TRUE!")

## Effect of condition and mode on in-game MS ratings (total score)

In [ ]:
ms = pd.melt(df.reset_index(), id_vars=['ID', 'condition'], value_vars=['H_AVG_MS', 'FP_AVG_MS'])
ms = ms.rename(columns={"variable" : 'mode', "value" : "rating"})
ms.replace({'mode': {'H_AVG_MS': 'Hybrid', 'FP_AVG_MS': 'First Person'},
               'condition': {1: 'Blob', 0: 'Avatar'}}, inplace=True)

ms_anova = pg.mixed_anova(dv='rating', between='condition', within='mode', subject='ID', data=ms)
ms_anova

In [ ]:
summarize(ms, "rating")

In [ ]:
boxplot(ms, "rating", "MS total score")

In [ ]:
# if interaction is significant, create interaction plot
if ms_anova["p-unc"].tolist()[-1] < sig_level:
    interact_plot(ms, "rating", "MS Total Score")

## Effect of condition and mode on SSQ (total score)

In [ ]:
ssq = pd.melt(df.reset_index(), id_vars=['ID', 'condition'], value_vars=['SSQ_TS_H', 'SSQ_TS_FP'])
ssq = ssq.rename(columns={"variable" : 'mode', "value" : "score"})
ssq.replace({'mode': {'SSQ_TS_H': 'Hybrid', 'SSQ_TS_FP': 'First Person'},
               'condition': {1: 'Blob', 0: 'Avatar'}}, inplace=True)

ssq_anova = pg.mixed_anova(dv='score', between='condition', within='mode', subject='ID', data=ssq)
ssq_anova

In [ ]:
summarize(ssq, "score")

In [ ]:
boxplot(ssq, "score", "SSQ total score")

In [ ]:
# if interaction is significant, create interaction plot
if ssq_anova["p-unc"].tolist()[-1] < sig_level:
    interact_plot(ms, "rating", "MS Total Score")

## Effect of condition and mode on in-game MS ratings (per area)

### Area 1

In [ ]:
area_one = pd.melt(df.reset_index(), id_vars=['ID', 'condition'], value_vars=['H_1_MS', 'FP_1_MS'])
area_one = area_one.rename(columns={"variable" : 'mode', "value" : "rating"})
area_one.replace({'mode': {'H_1_MS': 'Hybrid', 'FP_1_MS': 'First Person'},
               'condition': {1: 'Blob', 0: 'Avatar'}}, inplace=True)

area_one_anova = pg.mixed_anova(dv='rating', between='condition', within='mode', subject='ID', data=area_one)
area_one_anova

In [ ]:
summarize(area_one, "rating")

In [ ]:
# if interaction is significant, create interaction plot
if area_one_anova["p-unc"].tolist()[-1] < sig_level:
    interact_plot(area_one, "rating", "Interaction Condition x Mode (Area 1)")

### Area 2

In [ ]:
area_two = pd.melt(df.reset_index(), id_vars=['ID', 'condition'], value_vars=['H_2_MS', 'FP_2_MS'])
area_two = area_two.rename(columns={"variable" : 'mode', "value" : "rating"})
area_two.replace({'mode': {'H_2_MS': 'Hybrid', 'FP_2_MS': 'First Person'},
               'condition': {1: 'Blob', 0: 'Avatar'}}, inplace=True)

area_two_anova = pg.mixed_anova(dv='rating', between='condition', within='mode', subject='ID', data=area_two)
area_two_anova

In [ ]:
summarize(area_two, "rating")

In [ ]:
# if interaction is significant, create interaction plot
if area_two_anova["p-unc"].tolist()[-1] < sig_level:
    interact_plot(area_two, "rating", "Interaction Condition x Mode (Area 2)")

### Area 3

In [ ]:
area_three = pd.melt(df.reset_index(), id_vars=['ID', 'condition'], value_vars=['H_3_MS', 'FP_3_MS'])
area_three = area_three.rename(columns={"variable" : 'mode', "value" : "rating"})
area_three.replace({'mode': {'H_3_MS': 'Hybrid', 'FP_3_MS': 'First Person'},
               'condition': {1: 'Blob', 0: 'Avatar'}}, inplace=True)

area_three_anova = pg.mixed_anova(dv='rating', between='condition', within='mode', subject='ID', data=area_three)
area_three_anova

In [ ]:
summarize(area_three, "rating")

In [ ]:
# if interaction is significant, create interaction plot
if area_three_anova["p-unc"].tolist()[-1] < sig_level:
    interact_plot(area_three, "rating", "Interaction Condition x Mode (Area 3)")

### Area 4

In [ ]:
area_four = pd.melt(df.reset_index(), id_vars=['ID', 'condition'], value_vars=['H_4_MS', 'FP_4_MS'])
area_four = area_four.rename(columns={"variable" : 'mode', "value" : "rating"})
area_four.replace({'mode': {'H_4_MS': 'Hybrid', 'FP_4_MS': 'First Person'},
               'condition': {1: 'Blob', 0: 'Avatar'}}, inplace=True)

area_four_anova = pg.mixed_anova(dv='rating', between='condition', within='mode', subject='ID', data=area_four)
area_four_anova

In [ ]:
summarize(area_four, "rating")

In [ ]:
# if interaction is significant, create interaction plot
if area_four_anova["p-unc"].tolist()[-1] < sig_level:
    interact_plot(area_four, "rating", "Interaction Condition x Mode (Area 4)")

### Plot ratings for each area

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15,10), sharey=True)
plt.subplots_adjust(hspace=0.4)
fig.suptitle('Motion Sickness Ratings', fontsize=16)

ax = ax.ravel()

boxplot(area_one, "rating", "Area 1", ax[0])
boxplot(area_two, "rating", "Area 2", ax[1])
boxplot(area_three, "rating", "Area 3", ax[2])
boxplot(area_four, "rating", "Area 4", ax[3])

## Effect of condition and mode on SSQ (subscales)
### Ocolumotor Subscale

In [ ]:
ocolumotor = pd.melt(df.reset_index(), id_vars=['ID', 'condition'], value_vars=['SSQ_O_H', 'SSQ_O_FP'])
ocolumotor = ocolumotor.rename(columns={"variable" : 'mode', "value" : "score"})
ocolumotor.replace({'mode': {'SSQ_O_H': 'Hybrid', 'SSQ_O_FP': 'First Person'},
               'condition': {1: 'Blob', 0: 'Avatar'}}, inplace=True)

ocolumotor_anova = pg.mixed_anova(dv='score', between='condition', within='mode', subject='ID', data=ocolumotor)
ocolumotor_anova

In [ ]:
summarize(ocolumotor, "score")

In [ ]:
# if interaction is significant, create interaction plot
if ocolumotor_anova["p-unc"].tolist()[-1] < sig_level:
    interact_plot(ocolumotor, "score", "Interaction Condition x Mode (SSQ Ocolumotor Subscale)")

### Disorientation Subscale

In [ ]:
disorientation = pd.melt(df.reset_index(), id_vars=['ID', 'condition'], value_vars=['SSQ_D_H', 'SSQ_D_FP'])
disorientation = disorientation.rename(columns={"variable" : 'mode', "value" : "score"})
disorientation.replace({'mode': {'SSQ_D_H': 'Hybrid', 'SSQ_D_FP': 'First Person'},
               'condition': {1: 'Blob', 0: 'Avatar'}}, inplace=True)

disorientation_anova = pg.mixed_anova(dv='score', between='condition', within='mode', subject='ID', data=disorientation)
disorientation_anova

In [ ]:
summarize(disorientation, "score")

In [ ]:
# if interaction is significant, create interaction plot
if disorientation_anova["p-unc"].tolist()[-1] < sig_level:
    interact_plot(disorientation, "score", "Interaction Condition x Mode (SSQ Disorientation Subscale)")

### Nausea Subscale

In [ ]:
nausea = pd.melt(df.reset_index(), id_vars=['ID', 'condition'], value_vars=['SSQ_N_H', 'SSQ_N_FP'])
nausea = nausea.rename(columns={"variable" : 'mode', "value" : "score"})
nausea.replace({'mode': {'SSQ_N_H': 'Hybrid', 'SSQ_N_FP': 'First Person'},
               'condition': {1: 'Blob', 0: 'Avatar'}}, inplace=True)

nausea_anova = pg.mixed_anova(dv='score', between='condition', within='mode', subject='ID', data=nausea)
nausea_anova

In [ ]:
summarize(nausea, "score")

In [ ]:
# if interaction is significant, create interaction plot
if nausea_anova["p-unc"].tolist()[-1] < sig_level:
    interact_plot(nausea, "score", "Interaction Condition x Mode (SSQ Nausea Subscale)")

### Plot scores for each subscale

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,4), sharey=True)
plt.subplots_adjust(wspace=0.4)
fig.suptitle('SSQ Subscale Scores', fontsize=16, y=1.05)

ax = ax.ravel()

boxplot(ocolumotor, "score", "Ocolumotor", ax[0])
boxplot(disorientation, "score", "Disorientation", ax[1])
boxplot(nausea, "score", "Nausea", ax[2])

## Assumption check

In [ ]:
check_assumptions(ms, 'rating', "MS Total Score")

In [ ]:
check_assumptions(area_one, 'rating', "MS ratings Area 1")

In [ ]:
check_assumptions(area_two, 'rating', "MS ratings Area 2")

In [ ]:
check_assumptions(area_three, 'rating', "MS ratings Area 3")

In [ ]:
check_assumptions(area_four, 'rating', "MS ratings Area 4")

In [ ]:
check_assumptions(ssq, 'score', "SSQ Total Score")

In [ ]:
check_assumptions(ocolumotor, 'score', "SSQ Ocolumotor Subscale")

In [ ]:
check_assumptions(disorientation, 'score', "SSQ Disorientation Subscale")

In [ ]:
check_assumptions(nausea, 'score', "SSQ Nausea Subscale")

## Correlations of the motion sickness variables

In [ ]:
def corr_sig(df):
    '''
    Creates and returns a matrix containing the p-values of the spearman correlations within the df.
    '''
    p_matrix = np.zeros(shape=(df.shape[1],df.shape[1]))
    for col in df.columns:
        for col2 in df.drop(col,axis=1).columns:
            _ , p = stats.spearmanr(df[col],df[col2])
            p_matrix[df.columns.to_list().index(col),df.columns.to_list().index(col2)] = p
    return p_matrix

In [ ]:
df_corr = df[['SSQ_N_FP', 'SSQ_O_FP', 'SSQ_D_FP', 'SSQ_TS_FP',
              'SSQ_N_H', 'SSQ_O_H', 'SSQ_D_H', 'SSQ_TS_H', 
              'H_0_MS', 'H_1_MS', 'H_2_MS', 'H_3_MS', 'H_4_MS', 'H_AVG_MS', 
              'FP_0_MS', 'FP_1_MS', 'FP_2_MS', 'FP_3_MS', 'FP_4_MS', 'FP_AVG_MS']]

corrMatrix = df_corr.corr(method='spearman') # spearman because we have ordinal data
sigMatrix = corr_sig(df_corr)


# generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.set_theme(style="white")

### All correlations

In [ ]:
# generate a mask for the upper triangle
mask = np.zeros_like(corrMatrix, dtype=bool)
mask[np.triu_indices_from(mask)] = True

# draw the heatmap
f, ax = plt.subplots(figsize=(11, 9))
sns.heatmap(corrMatrix, mask=mask, cmap=cmap, vmin =0, vmax=1, center=0, 
              square=True, linewidths=.5, cbar_kws={"shrink": .5}, ax=ax,
              annot=True)

### Signficiant correlations

In [ ]:
# heatmap containing ONLY the significant correlations
mask = np.invert(np.tril(sigMatrix<0.05)) 

# draw the heatmap
f, ax = plt.subplots(figsize=(11, 9))
sns.heatmap(corrMatrix, mask=mask, cmap=cmap, vmin =0, vmax=1, center=0, 
              square=True, linewidths=.5, cbar_kws={"shrink": .5}, ax=ax,
              annot=True)